<a href="https://colab.research.google.com/github/ebi19912/AI/blob/main/Fingerprint_CNN_Persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# نصب کتابخانه kaggle
!pip install kaggle

# آپلود فایل kaggle.json
from google.colab import files
files.upload()

# تنظیمات مربوط به فایل kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# دانلود مجموعه داده از Kaggle
!kaggle datasets download -d peace1019/fingerprint-dataset-for-fvc2000-db4-b

# استخراج فایل‌ها از فایل فشرده
import zipfile
with zipfile.ZipFile('fingerprint-dataset-for-fvc2000-db4-b.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


In [ ]:
# نصب کتابخانه kaggle
!pip install kaggle

# آپلود فایل kaggle.json
from google.colab import files
files.upload()

# تنظیمات مربوط به فایل kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# دانلود مجموعه داده از Kaggle
!kaggle datasets download -d peace1019/fingerprint-dataset-for-fvc2000-db4-b

# استخراج فایل‌ها از فایل فشرده
import zipfile
with zipfile.ZipFile('fingerprint-dataset-for-fvc2000-db4-b.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers


تابع load_images_from_folder() مجموعه ای از تصاویر را از یک پوشه بارگیری می کند و یک لیست از تصاویر و یک لیست از برچسب ها را برمی گرداند. تابع در پوشه تکرار می شود و هر تصویر را با استفاده از تابع cv2.imread() بارگیری می کند. تابع همچنین برچسب هر تصویر را از نام فایل استخراج می کند. برچسب ها در یک لیست ذخیره می شوند.

توضیحات:

خط 1: تابع load_images_from_folder() تعریف می شود.

خط 2: یک لیست خالی برای ذخیره تصاویر ایجاد می شود.

خط 3: یک لیست خالی برای ذخیره برچسب ها ایجاد می شود.

خط 4: تابع os.listdir() برای دریافت لیست تمام فایل ها در پوشه مشخص شده استفاده می شود.

خط 5: تابع cv2.imread() برای بارگیری تصویر در مسیر مشخص شده استفاده می شود.

خط 6: عبارت if بررسی می کند که تصویر None نیست. اگر اینطور نیست، تصویر و برچسب به لیست های مربوطه اضافه می شوند.

خط 7: برچسب از نام فایل استخراج می شود. نام فایل بر روی کاراکتر '_' تقسیم می شود و اولین عنصر گرفته می شود. این عنصر سپس به یک عدد صحیح تبدیل می شود.

خط 8: عبارت return لیست های تصاویر و برچسب ها را برمی گرداند.



In [ ]:
# بارگیری و پردازش تصاویر
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            labels.append(int(filename.split('_')[0].split('.')[0]))  # استخراج برچسب از نام فایل
    return images, labels

x_train, y_train = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/train_data')
x_test, y_test = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/real_data')

print("Number of training images:", len(x_train))
print("Shape of the first training image:", x_train[0].shape)



این کد دو عملیات را انجام می دهد:

تبدیل لیست های تصاویر و برچسب ها به آرایه های NumPy
نرمال سازی تصاویر
تبدیل لیست ها به آرایه های NumPy:

تابع np.array() یک لیست را به یک آرایه NumPy تبدیل می کند. پارامتر -1 به تابع np.array() می گوید که تعداد ردیف ها را خود محاسبه کند.

نرمال سازی تصاویر:

تصاویر را می توان با تقسیم آنها بر 255.0 نرمال کرد. این کار باعث می شود که تمام مقادیر تصویر بین 0 و 1 قرار گیرند.

تبدیل برچسب ها به فرمت one-hot:

برچسب ها در مجموعه داده اثر انگشت FVC2000 DB4B اعداد صحیح هستند. برای استفاده از یک مدل شبکه عصبی، باید برچسب ها را به فرمت one-hot تبدیل کرد.

تابع to_categorical() یک آرایه از برچسب ها را به یک آرایه one-hot تبدیل می کند. در این مورد، هر برچسب به یک آرایه 10 بعدی تبدیل می شود که تنها عنصر غیر صفر آن در موقعیت برچسب مربوطه قرار دارد.

مثال:

فرض کنید لیست y_train حاوی برچسب های زیر باشد:

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

پس از اجرای کد زیر، لیست y_train به آرایه زیر تبدیل می شود:

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 ...,
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]


In [ ]:

# تبدیل لیست‌ها به آرایه‌های numpy و نرمال‌سازی
x_train = np.array(x_train).reshape(-1, 160, 160, 1).astype('float32') / 255.0
x_test = np.array(x_test).reshape(-1, 160, 160, 1).astype('float32') / 255.0


# تبدیل برچسب‌ها به فرمت one-hot
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



ایجاد شبکه عصبی

تابع models.Sequential() یک مدل شبکه عصبی ترتیبی ایجاد می کند. پارامتر input_shape شکل ورودی شبکه عصبی را مشخص می کند. در این مورد، شکل ورودی (160, 160, 1) است، زیرا تصاویر ورودی دارای ابعاد 160 در 160 پیکسل و تک کاناله هستند (تصاویر سیاه و سفید).

بدنه شبکه عصبی از یک سری لایه ها تشکیل شده است. هر لایه یک عمل خاصی را روی ورودی خود انجام می دهد و خروجی خود را به لایه بعدی ارسال می کند.

لایه های استفاده شده در این شبکه عصبی عبارتند از:

** لایه کانولوشن (Conv2D)**: این لایه ویژگی ها را از تصاویر ورودی استخراج می کند.
لایه استخرش (Pooling2D): این لایه ابعاد تصاویر را کاهش می دهد و در عین حال ویژگی های مهم را حفظ می کند.
لایه صاف (Flatten): این لایه ورودی را به یک بردار تبدیل می کند.
لایه متصل (Dense): این لایه یک تبدیل خطی را روی ورودی خود انجام می دهد.
آموزش شبکه عصبی

تابع model.compile() کامپایلر مدل را با مشخص کردن تابع بهینه ساز، تابع زیان و معیارهای ارزیابی تنظیم می کند.

تابع model.fit() مدل را روی داده های آموزش آموزش می دهد. پارامتر epochs تعداد دفعاتی را که مدل روی داده های آموزش تکرار می شود مشخص می کند.

ارزیابی شبکه عصبی

تابع model.evaluate() عملکرد مدل را روی داده های تست ارزیابی می کند. پارامتر verbose=0 باعث می شود که تابع هیچ خروجی چاپ نکند.



In [ ]:

# ایجاد شبکه عصبی
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(160, 160, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])


# آموزش شبکه عصبی
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

# ارزیابی شبکه عصبی
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Saving kaggle.json to kaggle.json
100% 25.9M/25.9M [00:01<00:00, 21.5MB/s]
100% 25.9M/25.9M [00:01<00:00, 14.0MB/s]
Number of training images: 800
Shape of the first training image: (160, 160)
Epoch 1/10
25/25 [==============================] - 31s 1s/step - loss: 3.4536 - accuracy: 0.0950
Epoch 2/10
25/25 [==============================] - 30s 1s/step - loss: 2.3020 - accuracy: 0.1075
Epoch 3/10
25/25 [==============================] - 30s 1s/step - loss: 2.2813 - accuracy: 0.1650
Epoch 4/10
25/25 [==============================] - 30s 1s/step - loss: 2.0426 - accuracy: 0.3225
Epoch 5/10
25/25 [==============================] - 30s 1s/step - loss: 1.3363 - accuracy: 0.5888
Epoch 6/10
25/25 [==============================] - 31s 1s/step - loss: 0.8474 - accuracy: 0.7312
Epoch 7/10
25/25 [==============================] - 30s 1s/step - loss: 0.6008 - accuracy: 0.8150
Epoch 8/10
25/25 [==============================] - 31s 1s/step - loss: 0.4218 - accuracy: 0.9000
Epoch 9/10
25/25 [=====

ValueError: ignored

In [ ]:
!pip install joblib
import joblib


ثبت اثر انگشت

تابع register_fingerprint() یک اثر انگشت جدید را ثبت می کند. این تابع ابتدا تصویر اثر انگشت را پیش پردازش می کند و آن را به فرمت مناسب برای مدل شبکه عصبی تبدیل می کند. سپس، ویژگی های اثر انگشت را با استفاده از مدل شبکه عصبی پیش بینی می کند. در نهایت، ویژگی های اثر انگشت را ذخیره می کند.

مراحل ثبت اثر انگشت:

پیش پردازش تصویر:

تصویر را به حالت سیاه و سفید تبدیل کنید.
تصویر را به اندازه 160 در 160 پیکسل تغییر اندازه دهید.
تصویر را به یک آرایه NumPy با شکل (-1, 160, 160, 1) تبدیل کنید.
تصویر را نرمال کنید تا تمام مقادیر آن بین 0 و 1 قرار گیرند.
پیش بینی ویژگی های اثر انگشت:

ویژگی های اثر انگشت را با استفاده از مدل شبکه عصبی پیش بینی کنید.
ویژگی های اثر انگشت را به یک بردار تبدیل کنید.
ذخیره ویژگی های اثر انگشت:

ویژگی های اثر انگشت را در یک فایل با فرمت pickle ذخیره کنید.

In [ ]:
# ثبت اثر انگشت
def register_fingerprint(image):
    # پیش پردازش تصویر
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # پیش بینی ویژگی‌های اثر انگشت
    features = model.predict(image)

    # ذخیره ویژگی‌ها
    features = features.flatten()
    with open('fingerprint_features.pickle', 'wb') as f:
        joblib.dump(features, f)

# احراز هویت اثر انگشت
def authenticate_fingerprint(image):
    # پیش پردازش تصویر
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # پیش بینی ویژگی‌های اثر انگشت
    features = model.predict(image)

    # مقایسه ویژگی‌ها
    with open('fingerprint_features.pickle', 'rb') as f:
        known_features = joblib.load(f)

    # استفاده از تابع فاصله برای مقایسه ویژگی‌ها
    dist = np.linalg.norm(features - known_features, axis=1)

    # نتیجه احراز هویت
    if dist < 0.01:
        return True
    else:
        return False


1/1 [==============================] - 0s 33ms/step
True



خط اول تصویر `FingerPrintOriginal.jpg` را در متغیر `image` ذخیره می کند. خط دوم اثر انگشت را با استفاده از تابع `register_fingerprint()` ثبت می کند. خط سوم تصویر `FingerPrintOriginal.jpg` را دوباره در متغیر `image` ذخیره می کند. خط چهارم اثر انگشت را با استفاده از تابع `authenticate_fingerprint()` احراز هویت می کند و نتیجه را در متغیر `is_authenticated` ذخیره می کند. خط پنجم متغیر `is_authenticated` را چاپ می کند.

اگر اثر انگشت معتبر باشد، خروجی کد شما `True` خواهد بود. در غیر این صورت، خروجی کد شما `False` خواهد بود.

برای اجرای این کد، ابتدا باید کتابخانه های `cv2` و `joblib` را نصب کنید. سپس، می توانید کد را در یک فایل پایتون با پسوند `.py` ذخیره کنید و آن را با استفاده از کامند زیر اجرا کنید:

```
python fingerprint_authentication.py
```

اگر اثر انگشت معتبر باشد، خروجی کد به صورت زیر خواهد بود:

```
True
```

در غیر این صورت، خروجی کد به صورت زیر خواهد بود:

```
False
```

In [ ]:

# ثبت اثر انگشت
image = cv2.imread('FingerPrintOriginal.jpg')
register_fingerprint(image)

# احراز هویت اثر انگشت
image = cv2.imread('FingerPrintOriginal.jpg')
is_authenticated = authenticate_fingerprint(image)
print(is_authenticated)
